In [1]:
import sys
from PyQt5.QtWidgets import QApplication, QMainWindow, QTextEdit, QLineEdit, QPushButton, QVBoxLayout, QWidget, QInputDialog
from PyQt5.QtCore import QThread, pyqtSignal
import socket

In [2]:

class ClientThread(QThread):
    message_received = pyqtSignal(str)

    def __init__(self, username, host='localhost', port=12345):
        super().__init__()
        self.client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.client_socket.connect((host, port))
        self.client_socket.send(username.encode('utf-8'))
        self.username = username
    
    def run(self):
        while True:
            try:
                message = self.client_socket.recv(1024).decode('utf-8')
                if message:
                    self.message_received.emit(message)
            except:
                self.client_socket.close()
                break
    
    def send_message(self, message):
        self.client_socket.send(message.encode('utf-8'))

class ChatClient(QMainWindow):
    def __init__(self, host='localhost', port=12345):
        super().__init__()
        self.username, ok = QInputDialog.getText(self, 'Username', 'Enter your username:')
        if ok:
            self.initUI()
            self.client_thread = ClientThread(self.username, host, port)
            self.client_thread.message_received.connect(self.receive_message)
            self.client_thread.start()

    def initUI(self):
        self.setGeometry(300, 300, 400, 500)
        self.setWindowTitle(f'Chat Client - {self.username}')
        
        self.chat_display = QTextEdit(self)
        self.chat_display.setReadOnly(True)
        
        self.message_input = QLineEdit(self)
        self.message_input.setPlaceholderText('Type your message here...')
        
        self.send_button = QPushButton('Send', self)
        self.send_button.clicked.connect(self.send_message)
        
        layout = QVBoxLayout()
        layout.addWidget(self.chat_display)
        layout.addWidget(self.message_input)
        layout.addWidget(self.send_button)
        
        container = QWidget()
        container.setLayout(layout)
        
        self.setCentralWidget(container)
        self.show()
    
    def send_message(self):
        message = self.message_input.text()
        if message:
            self.client_thread.send_message(message)
            self.message_input.clear()
            self.chat_display.append(f"You: {message}")
    
    def receive_message(self, message):
        if message.startswith(f"{self.username}:"):
            # Display it as 'You' if the message is from the current user
            self.chat_display.append(message.replace(f"{self.username}:", "You:"))
        else:
            # Message from another user
            self.chat_display.append(message)

# Run the client
app = QApplication(sys.argv)
client = ChatClient()
sys.exit(app.exec_())


SystemExit: 0

C:\Users\subha\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
